In [1]:
import numpy as np
from time import time
import ast
import label_utils as lu
import logging

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import LinearSVC

In [ ]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
log_path = '/home/pogorelov/work/svm.log'
logging.basicConfig(filename=log_path, filemode="w", level=logging.INFO)

In [2]:
all_labels_file = 'all_labels.txt'
embeddings = 'gexf_graphs_dims_300_epochs_30_lr_0.3_embeddings.txt'
X, Y = lu.X_Y_from_embeddings(all_labels_file, embeddings)

In [7]:
c0,c1,c2=0,0,0
for item in Y:
    if item == 0:
        c0 += 1
    if item == 1:
        c1 += 1
    if item == 2:
        c2 += 1

print('clean %.1f, pua %.1f, malw %.1f' % ((c0/(c0+c1+c2)), (c1/(c0+c1+c2)), (c2/(c0+c1+c2))))

clean 0.5, pua 0.3, malw 0.2


In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=12345)
logging.info('Train and Test matrix shapes: {}, {}, {}, {} '.format(X_train.shape, X_test.shape,
                                                                        Y_train.shape, Y_test.shape))
# params = {'C':[0.01,0.1,1,10,100,1000]}
# params = {'C':[0.01,0.1,1]}
params = {'C':[0.1]}
classifier = GridSearchCV(LinearSVC(), params, scoring='accuracy',verbose=1)
start = time()
classifier.fit(X_train,Y_train)
end = time()
logging.info('fit time is %.1f min' % ((end-start)/60))
# logging.info('best classifier model\'s hyperparamters', classifier.best_params_)

Y_pred = classifier.predict(X_test)

acc = accuracy_score(Y_test, Y_pred)
logging.info('Linear SVM accuracy: {}'.format(acc))

logging.info(classification_report(Y_test, Y_pred))